# EDA of Vehicle Sales Advertisement Data Set

The following is an exploratory data analysis notebook.  
The data set is of car sales advertisements.  
Insights and conclusions will be explored with the ultimate goal of creating an interactive web application to make use of the data set for potential buyers, sellers, or analysts interested in the used car market.

In [3]:
# Import standard libraries
import sys
from pathlib import Path

# Add the parent directory to sys.path
sys.path.append(Path(
    '/Volumes/T7/Coding_Files/TripleTen Data Science/Sprint 4/Used Vehicle Sales App').resolve().as_posix())

# Import third-party libraries
import plotly.express as px
import numpy as np
import pandas as pd
import streamlit as st

# Import local modules
import functions as my_funcs

## First Look

In [43]:
# Read the data
vehicles_df = pd.read_csv('../vehicles_us.csv')

# Look for missing values and get info
print(vehicles_df.isna().sum())
print()
vehicles_df.info()

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-nul

In [44]:
# Get duplicate row count
print(vehicles_df.duplicated().sum())

0


### Price Column

In [45]:
vehicles_df['price'].describe()

count     51525.000000
mean      12132.464920
std       10040.803015
min           1.000000
25%        5000.000000
50%        9000.000000
75%       16839.000000
max      375000.000000
Name: price, dtype: float64

In [46]:
# Hmmm, there are some vehicles with a price of 1.
print(f'Vehicle price $1 or less:', len(
    vehicles_df[vehicles_df['price'] <= 1]))
print()

# Any pattern here with the model?
print(vehicles_df[vehicles_df['price'] <= 1]['model'].value_counts())

Vehicle price $1 or less: 798

model
dodge charger           76
ram 3500                76
ford f-150              60
jeep wrangler           55
chevrolet camaro        51
                        ..
ford f250 super duty     1
dodge grand caravan      1
nissan sentra            1
nissan murano            1
gmc sierra 2500hd        1
Name: count, Length: 61, dtype: int64


I'm not really seeing a pattern. At first glance it seemed odd to me that a car would be advertised for $1, but I think that is perfectly fine. I don't think there is a need to change anything in this column.

### Model Year Column

In [47]:
# DF for missing model year
missing_model_year = vehicles_df[vehicles_df['model_year'].isna()]
# value counts for model
missing_model_year['model'].value_counts()

model
ford f-150                           191
chevrolet silverado 1500             155
ram 1500                             117
chevrolet silverado                   98
honda accord                          80
                                    ... 
kia sorento                           14
hyundai santa fe                      13
acura tl                              12
honda civic lx                        11
mercedes-benz benze sprinter 2500      7
Name: count, Length: 100, dtype: int64

I'm not seeing any sort of pattern with the missing values. I think I will just have to accept that these will be missing values given the nature of the dataset. Same can be said for all the other columns with missing data. 

I think I can move on to asking questions about the data at this point and see if I run into any unexpected discrepancies.

In [48]:
# Delete missing_model_year df for memory optimization
del missing_model_year

### Model Column

In [49]:
print(sorted(vehicles_df['model'].unique().tolist()))

['acura tl', 'bmw x5', 'buick enclave', 'cadillac escalade', 'chevrolet camaro', 'chevrolet camaro lt coupe 2d', 'chevrolet colorado', 'chevrolet corvette', 'chevrolet cruze', 'chevrolet equinox', 'chevrolet impala', 'chevrolet malibu', 'chevrolet silverado', 'chevrolet silverado 1500', 'chevrolet silverado 1500 crew', 'chevrolet silverado 2500hd', 'chevrolet silverado 3500hd', 'chevrolet suburban', 'chevrolet tahoe', 'chevrolet trailblazer', 'chevrolet traverse', 'chrysler 200', 'chrysler 300', 'chrysler town & country', 'dodge charger', 'dodge dakota', 'dodge grand caravan', 'ford econoline', 'ford edge', 'ford escape', 'ford expedition', 'ford explorer', 'ford f-150', 'ford f-250', 'ford f-250 sd', 'ford f-250 super duty', 'ford f-350 sd', 'ford f150', 'ford f150 supercrew cab xlt', 'ford f250', 'ford f250 super duty', 'ford f350', 'ford f350 super duty', 'ford focus', 'ford focus se', 'ford fusion', 'ford fusion se', 'ford mustang', 'ford mustang gt coupe 2d', 'ford ranger', 'ford 

#### Issues
1. ford f-number vs fnumber
2. ford f-number super duty vs sd

##### Approach
look for 'ford f', if next character is a number add '-' between 'f' and the number
Then check if last two characters are 'sd', if so, change to 'super duty'.  

Then capitalize the first letter of each 'word'. This should be done all before splitting into make and model

### Fuel Column

In [50]:
# Are there models with multiple fuel types?
vehicles_df.groupby(['model', 'model_year'])[
    'fuel'].nunique().sort_values(ascending=False)

model                model_year
honda civic          2007.0        4
ford f250            2013.0        3
                     1997.0        3
chevrolet silverado  2008.0        3
                     2011.0        3
                                  ..
ford f-250 sd        2007.0        1
                     2004.0        1
                     2001.0        1
                     2000.0        1
volkswagen passat    2018.0        1
Name: fuel, Length: 2226, dtype: int64

### Cylinders Column

In [51]:
print(vehicles_df['cylinders'].value_counts(dropna=False))

cylinders
8.0     15844
6.0     15700
4.0     13864
NaN      5260
10.0      549
5.0       272
3.0        34
12.0        2
Name: count, dtype: int64


Oh, Cylinders is float type because it contains NaN.

### Odometer Column

In [52]:
# Are there any floats in the odometer column?
Odometer_na_droped = vehicles_df.dropna(subset=['odometer'])

# Check if there are any floats in the odometer column using the numpy array_equal method
if np.array_equal(Odometer_na_droped['odometer'].values, Odometer_na_droped['odometer'].values.astype(int)):
    print('No floats in the odometer column')
else:
    print('There are floats in the odometer column')

No floats in the odometer column


# Temp Transformations

In [33]:
# Create a make column by splitting the model column
vehicles_df['make'] = vehicles_df['model'].str.split(' ').str[0]

# Remove the make from the model column
vehicles_df['model'] = vehicles_df['model'].str.split(
    ' ').str[1:].str.join(' ')

# Rearrange the columns
vehicles_df = vehicles_df[['price', 'make', 'model', 'model_year', 'condition', 'cylinders', 'fuel',
                           'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
                           'date_posted', 'days_listed']]

# Rename model_year to year
vehicles_df.rename(columns={'model_year': 'year'}, inplace=True)

## Explore Plots and Visualizations

In [34]:
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   make          51525 non-null  object 
 2   model         51525 non-null  object 
 3   year          47906 non-null  float64
 4   condition     51525 non-null  object 
 5   cylinders     46265 non-null  float64
 6   fuel          51525 non-null  object 
 7   odometer      43633 non-null  float64
 8   transmission  51525 non-null  object 
 9   type          51525 non-null  object 
 10  paint_color   42258 non-null  object 
 11  is_4wd        25572 non-null  float64
 12  date_posted   51525 non-null  object 
 13  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(8)
memory usage: 5.5+ MB


### What are some interactive questions I can ask about this dataset?
Note: USED CAR SALES

1. What is the sales price distribution for each model in the dataset?

2. Scatter plot sale price vs odometer with check mark for fuel type detail?

3.  What is the average sale price per model year for selected model?

#### Sale Price Distribution per Model

In [35]:
# Plot the distribution of the price for the given make, model, year, and fuel type
make = 'toyota'
model = 'camry'
year = 2010
fuel = 'gas'

# Create mask
df_mask = (vehicles_df['make'] == make) & (
    vehicles_df['model'] == model) & (vehicles_df['year'] == year) & (vehicles_df['fuel'] == fuel)

# Filter the data
filtered_df = vehicles_df[df_mask]

# Plot the distribution of the price
px.histogram(filtered_df, x='price', histnorm='percent',
             color='condition', template='plotly')

In [36]:
# Plot the scatter for the given make, model, year, and condition

# Vehicle specifications
make = 'honda'
model = 'civic'
year = 2007
condition = 'excellent'

# Create mask to filter data
df_mask = (vehicles_df['make'] == make) & (vehicles_df['model'] == model) & (
    vehicles_df['year'] == year) & (vehicles_df['condition'] == condition)

# Filter the data
filtered_df = vehicles_df[df_mask]
filtered_df = filtered_df.dropna(
    subset=['price', 'odometer'])  # Ensure no NaN values

# Calculate slope and intercept for linear regression
x = filtered_df['odometer']
y = filtered_df['price']
slope, intercept = np.polyfit(x, y, 1)

# Create the scatter plot
fig = px.scatter(filtered_df, x='odometer', y='price',
                 trendline='ols', template='plotly', color='fuel')

# Annotation text, converting the slope to a string and formatting it to show as dollars per 1k miles
slope_annotation = f"Depreciation Rate: ${(slope * 1000):.2f} per 1K miles"

# Add annotation to the figure, adjusted to the top right
fig.add_annotation(x=1, y=1, text=slope_annotation,
                   xref="paper", yref="paper",  # Use 'paper' to position relative to the plot area
                   showarrow=False, align='right')

# Direction for App  

I envision this as a tool for car buyers, so here are the plots / info I like best:  
1. Sales price distribution for a given make, model, year, and fuel type, check box for showing condition on hist plot.
2. Scatter plot for showing value vs odometer reading for a given make, model, year, and fuel type, check box for considering condition. (If then statement for building mask)

I like the optional consideration for condition because I have found that it is hard to find consensus on condition when car buying, so it sometimes muddies the water.

## Cleaning, Transformations, and Feature Engineering

In [37]:
# Now that I know what I am looking to accomplish with the application, I can set the data up for the app. This will be the initial setup for the app.

# Reload the data from the csv file for a fresh start
vehicles_df = pd.read_csv('../vehicles_us.csv')

# Keep only the necessary columns
keep_columns = ['price', 'model_year',
                'model', 'condition', 'fuel', 'odometer']
# ... by dropping the columns not in the keep_columns list
vehicles_df = vehicles_df[keep_columns]

# Drop rows missing the year since that will be a necessary filter
vehicles_df = vehicles_df.dropna(subset=['model_year'])
# Missing odometer can stay since they will be dropped by default when creating the scatter plot

# Strip and lowercase the string columns
string_columns = ['model', 'condition', 'fuel']
vehicles_df[string_columns] = vehicles_df[string_columns].apply(
    lambda x: x.str.strip().str.lower())

# Apply the normalize ford f series function to the model column
vehicles_df['model'] = vehicles_df['model'].apply(
    my_funcs.normalize_ford_f_series)

# Create make column by splitting the model column
vehicles_df['make'] = vehicles_df['model'].str.split(' ').str[0]
# Remove the make from the model column
vehicles_df['model'] = vehicles_df['model'].str.split(
    ' ').str[1:].str.join(' ')

# Rename columns to my liking
vehicles_df.rename(columns={'model_year': 'year', 'fuel': 'fuel_type',
                   'odometer': 'odometer_miles'}, inplace=True)

# Rearrange the columns to my liking
vehicles_df = vehicles_df[['price', 'make', 'model',
                           'year', 'condition', 'fuel_type', 'odometer_miles']]

# Change year to integer
vehicles_df['year'] = vehicles_df['year'].astype(int)

In [38]:


vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47906 entries, 0 to 51524
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   price           47906 non-null  int64  
 1   make            47906 non-null  object 
 2   model           47906 non-null  object 
 3   year            47906 non-null  int64  
 4   condition       47906 non-null  object 
 5   fuel_type       47906 non-null  object 
 6   odometer_miles  40563 non-null  float64
dtypes: float64(1), int64(2), object(4)
memory usage: 2.9+ MB
